In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os, math
from pprint import pprint  # pretty-printer
from collections import defaultdict
from gensim.models import CoherenceModel, LdaModel, LdaMulticore


In [2]:
filepath = "../../data/chengdu_taxi/"
filename = filepath + os.path.join(filepath, '20140803_train.txt')
dataset = pd.read_csv(filename, header=None, names=["ID", "lat", "lng", "passengers", "timestamp"])
print(dataset.head())
print(dataset.info())
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset['hour'] = dataset['timestamp'].dt.hour

   ID        lat         lng  passengers          timestamp
0   1  30.624806  104.136604           1  2014/8/3 21:18:46
1   1  30.624809  104.136612           1  2014/8/3 21:18:15
2   1  30.624811  104.136587           1  2014/8/3 21:20:17
3   1  30.624811  104.136596           1  2014/8/3 21:19:16
4   1  30.624811  104.136619           1  2014/8/3 21:17:44
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53045407 entries, 0 to 53045406
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID          int64  
 1   lat         float64
 2   lng         float64
 3   passengers  int64  
 4   timestamp   object 
dtypes: float64(2), int64(2), object(1)
memory usage: 2.0+ GB
None


# 清空变量内存

In [13]:
# import gc
# del [dataset]
# gc.collect()

505

# 划分区域 生成区域

In [3]:
# split gird
# 1经度111KM

def gird_by_span():
    #根据框个数划分
    column_num, row_num = 200, 200
    gird_lat_span = (LATMAX - LATMIN)/row_num
    gird_lng_span = (LNGMAX - LNGMIN)/column_num
    return gird_lat_span, gird_lng_span, column_num, row_num

def gird_by_distance(lng_dis = 0.5, lat_dis = 0.4):
    #根据距离划分
    #纬度跨度0.4km 经度跨度0.5km
    #经纬度距离估算https://blog.csdn.net/weixin_35301706/article/details/112527068

    gird_lat_span = 1 / (111 / lat_dis)
    gird_lng_span = 1 / ((111 * math.cos( (LATMAX + LATMIN) / 2)) / lng_dis)
    row_num = round((LATMAX - LATMIN) / gird_lat_span)
    column_num = round((LNGMAX - LNGMIN) / gird_lng_span)
    return gird_lat_span, gird_lng_span, column_num, row_num

LATMAX, LATMIN, LNGMAX, LNGMIN = np.max(dataset['lat']), np.min(dataset['lat']), np.max(dataset['lng']), np.min(dataset['lng'])
print(LATMAX, LATMIN, LNGMAX, LNGMIN)

# gird_lat_span, gird_lng_span, column_num, row_num = gird_by_span()
gird_lat_span, gird_lng_span, column_num, row_num = gird_by_distance()


def LngLat2GirdID(lat,lng):
    if lng < LNGMIN or lng > LNGMAX or lat < LATMIN or lat > LATMAX:
        return -1

    return int((lat-LATMIN)/gird_lat_span) + int((lng-LNGMIN)/gird_lng_span) * column_num 


def GirdID2LngLat(gird_id):
    curr_row = int(gird_id/row_num)
    curr_column = gird_id % column_num

    return [LATMIN + gird_lat_span * (curr_row + 0.5 ), LNGMIN + gird_lng_span * (curr_column + 0.5)]

#划分区域id

31.032468 30.290675 104.609693 103.269638


In [5]:
dataset['gird_id'] = dataset.apply(lambda x: LngLat2GirdID(x['lat'], x['lng']), axis=1)

# 生成用于[轨迹-小时-序列]，用于后续生成文档

In [6]:
hour_list = np.unique(dataset['hour'])
out_filename_part = '20140803_train_'
alltraSequence = pd.DataFrame(columns=['TraID', 'hour', 'sequence'])

for i, v in enumerate(hour_list):
    traSequence = []
    outfile = out_filename_part + str(v) + '.csv'
    dataset_sub = dataset.loc[dataset['hour'] == v] 
    dataset_sub = dataset_sub.groupby([dataset_sub['ID'], dataset_sub['hour']])
    
    for index, group in dataset_sub:
        group_sorted = group.sort_values(['timestamp'], ascending = [True])
        traSequence.append([index[0], index[1], group_sorted.gird_id.tolist()])
    
    traSequence = pd.DataFrame(traSequence, columns=['TraID', 'hour', 'sequence'])
    traSequence.to_csv(os.path.join(filepath, outfile), index=False)
    
    alltraSequence = pd.concat([alltraSequence, traSequence], ignore_index=True)
traSequence = alltraSequence

In [ ]:
traSequence.to_csv(os.path.join(filepath, '20140803_sequence_full.csv'), index=False) #保存在本地 下次不用再次执行上边代码

# 第二次运行时可以从这里开始
免去数据组织

In [11]:
#load data from disk
import pandas as pd
import numpy as np
import os, math
from pprint import pprint  # pretty-printer
from collections import defaultdict
from gensim.corpora.dictionary import Dictionary
import ast

filepath = "../../data/chengdu_taxi/"
#traSequence = pd.read_csv(os.path.join(filepath, '20140803_sequence_full.csv'))
traSequence = pd.read_csv(os.path.join(filepath, '20140803_train_6.csv'))
traSequence['sequence'] = traSequence['sequence'].apply(lambda x: ast.literal_eval(x))

# construct documents texts dicionary corpus

In [13]:
# construct documents
documents = []
for seq in zip(traSequence['sequence']):
    my_list = list(seq)[0]
    my_list_pair = []
    for idx, current in enumerate(my_list):
        if len(my_list) == 1:
            my_list_pair.append(str(current) + '-' + str(current))
        elif idx < len(my_list) - 1:
            my_list_pair.append(str(current) + '-' + str(my_list[idx + 1]))
    join_s = " "
    join_s = join_s.join(my_list_pair)
    documents.append(join_s)

# construct texts
# remove words that appear only once
texts = [[word for word in document.lower().split()] for document in documents]
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text] for text in texts]

## dictionary

dictionary = Dictionary(texts)
dictionary.save('./tmp/20140803_full.dict')  # store the dictionary, for future reference
corpus = [dictionary.doc2bow(text) for text in texts]
#corpus = [['id', 'count'], ['id', 'count']]

ldaseq运行非常慢

In [ ]:
from gensim.models import LdaSeqModel

#find hour corresponding list
hour_list = np.unique(traSequence.hour).tolist()
time_slice = [len(traSequence[traSequence['hour'] == hour]) for i, hour in enumerate(hour_list)]
print('construct ldaseq')
ldaseq = LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=18)

# dynamic topic modeling analysis
- https://markroxor.github.io/gensim/static/notebooks/ldaseqmodel.html#topic=0&lambda=0.83&term=
- Printing Topics
- Looking for Topic Evolution
- Document - Topic Proportions
- Distances between documents
- Chain Variance
- Topic Coherence for DTM

## Printing topics
Get the most relevant words for every topic.

In [8]:
pprint(ldaseq.print_topics(time=0))

[[('26132-26132', 0.061521670312400974),
  ('29621-29621', 0.0454871294496665),
  ('29171-29171', 0.042325964222196685),
  ('27648-27648', 0.035527153625599904),
  ('29381-29381', 0.0349851100116677),
  ('27006-27006', 0.02592361683863938),
  ('25257-25257', 0.020231574375552897),
  ('30690-30690', 0.01915808662129951),
  ('30259-30259', 0.016811006145362354),
  ('26566-26566', 0.016119299654327214),
  ('23077-23077', 0.01603159168876552),
  ('25720-25720', 0.015908223563896654),
  ('30041-30041', 0.01428315207489084),
  ('28954-28954', 0.014082602262864853),
  ('28953-28953', 0.012436506082785443),
  ('29609-29609', 0.011687890683442443),
  ('30475-30475', 0.011438114090627878),
  ('30258-30258', 0.010081291298189772),
  ('29825-29825', 0.009671757323385386),
  ('29174-29174', 0.009188892695132239)],
 [('29168-29168', 0.19304560961113235),
  ('29396-29396', 0.06857710871848666),
  ('25065-25065', 0.06140593429926787),
  ('28752-28752', 0.056810104325810525),
  ('28967-28967', 0.037087

## Looking for Topic Evolution
To fix a topic and see it evolve, use print_topic_times. 

In [10]:
pprint(ldaseq.print_topic_times(topic=0))

[[('9', 0.13409381204548854),
  ('10', 0.11466000998754254),
  ('11', 0.09137169443299875),
  ('2', 0.07918483111432532),
  ('7', 0.07907639383611106),
  ('5', 0.07826225346973789),
  ('4', 0.07351501667722589),
  ('6', 0.07344995495497192),
  ('1', 0.07179158725343673),
  ('3', 0.07066024824961405),
  ('8', 0.0670171042817487),
  ('0', 0.06691709369679869)],
 [('9', 0.13414804420701243),
  ('10', 0.11469024989834913),
  ('11', 0.09137715359074329),
  ('2', 0.07917937913054811),
  ('7', 0.07907085197803475),
  ('5', 0.0782560411237415),
  ('4', 0.0735050489984913),
  ('6', 0.07343993768384607),
  ('1', 0.07178032353389016),
  ('3', 0.07064815391671146),
  ('8', 0.06700244720381814),
  ('0', 0.06690236873481366)],
 [('9', 0.13419675564485226),
  ('10', 0.11471740630992798),
  ('11', 0.09138205246145793),
  ('2', 0.07917448083314886),
  ('7', 0.07906587299310248),
  ('5', 0.07825046019352967),
  ('4', 0.07349609687434866),
  ('6', 0.0734309410431438),
  ('1', 0.07177020804800968),
  ('3'

In [39]:
words = [dictionary[word_id] for word_id, count in corpus[1000]]
words

['25061-25061',
 '25061-25278',
 '25278-25278',
 '25507-25507',
 '25712-25495',
 '25712-25712',
 '25936-25936',
 '25936-25937',
 '25937-25936',
 '25937-25937',
 '25937-26154',
 '26154-25937',
 '26154-26154',
 '26154-26372',
 '26155-26154',
 '26370-26153',
 '26370-26370',
 '26372-26155',
 '26372-26372',
 '25060-25060',
 '25060-25061',
 '25278-25495',
 '25495-25712',
 '25277-25276',
 '25277-25277',
 '25278-25277',
 '25495-25278',
 '25071-24854',
 '25071-25071',
 '25072-25071',
 '25276-25276',
 '25279-25278',
 '25289-25072',
 '25289-25289',
 '25289-25290',
 '25498-25497',
 '25505-25289',
 '25716-25498',
 '25717-25716',
 '25719-25719',
 '25721-25505',
 '25721-25721',
 '25938-25938',
 '26590-26372',
 '26808-26590',
 '26809-26808',
 '27027-27027',
 '26153-26370',
 '25496-25497',
 '25497-25498',
 '25498-25716',
 '25939-25938',
 '25716-25718',
 '25278-25496',
 '25718-25936',
 '25276-25059',
 '25497-25279',
 '25720-25721',
 '25937-25720',
 '25059-25059',
 '25059-25060',
 '25070-25070',
 '25070-

Document - Topic Proportions
1000??

In [23]:
words = [dictionary[word_id] for word_id, count in corpus[1000]]
print (words)
doc = ldaseq.doc_topics(1000) # check the 558th document in the corpuses topic distribution
print (doc)

['25061-25061', '25061-25278', '25278-25278', '25507-25507', '25712-25495', '25712-25712', '25936-25936', '25936-25937', '25937-25936', '25937-25937', '25937-26154', '26154-25937', '26154-26154', '26154-26372', '26155-26154', '26370-26153', '26370-26370', '26372-26155', '26372-26372', '25060-25060', '25060-25061', '25278-25495', '25495-25712', '25277-25276', '25277-25277', '25278-25277', '25495-25278', '25071-24854', '25071-25071', '25072-25071', '25276-25276', '25279-25278', '25289-25072', '25289-25289', '25289-25290', '25498-25497', '25505-25289', '25716-25498', '25717-25716', '25719-25719', '25721-25505', '25721-25721', '25938-25938', '26590-26372', '26808-26590', '26809-26808', '27027-27027', '26153-26370', '25496-25497', '25497-25498', '25498-25716', '25939-25938', '25716-25718', '25278-25496', '25718-25936', '25276-25059', '25497-25279', '25720-25721', '25937-25720', '25059-25059', '25059-25060', '25070-25070', '25070-25071', '25071-25289', '25940-25940', '25938-26154', '26154-26